### 机器学习实战 Machine Learning in Action
#### 4. navie bayes

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
class myself_OneHotEncoder(object):
    def get_Dict(self, labels): #data is a list
        label_dict = {}
        #create a label dictionary
        for i,lable in enumerate(set(labels)):
            label_dict[lable] = i
        
        #create label exchange keys,values
        label_dict_reverse = dict((v,k) for k,v in label_dict.items())
        
        return label_dict,label_dict_reverse
    
    def get_OneHotEncoder(self,labels):
        # get labels to integer
        label_dict,_ = self.get_Dict(labels)
        integer_encoded = []
        for label in labels:
            integer_encoded.append(label_dict[label])
        
        # get integer labels to OneHot labels
        integer_encoded = np.array(integer_encoded)
        onehot_encoder = OneHotEncoder(sparse=False)
        integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
        onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
        return onehot_encoded
        

In [ ]:
import numpy as np
from collections import Counter
class bayes:
    
    def createVocabList(self,dataset):
        vocabSet = set([])
        for document in dataset:
            vocabSet = vocabSet | set(document)
        return list(vocabSet)
    
    def setofWord2Vec(self,vocabList,inputSet):
        returnVec = [0]*len(vocabList) #a list 
        for word in inputSet:
            if word in vocabList:
                returnVec[vocabList.index(word)] = 1
            else :
                print('the word: %s is not in my Vocabulary! ' %word)
        return returnVec
    
    #navie bayes classifier training
    def trainNB0(self,dataset, labels):   #labels type with onehot
        self.labels = labels
        self.vocabSetlist = self.createVocabList(dataset)
        train_Matrix = []
        for w in dataset: 
            train_Matrix.append(self.setofWord2Vec(self.vocabSetlist,w))
        print('train_Matrix',train_Matrix)
        numTrainDocs = len(train_Matrix)  #num of samples
        numWords = len(train_Matrix[0])  #num of features
        #self.pAbusive = sum(labels)/float(numTrainDocs) #样本为1的比例
        #mulity classifier
        '''
        #public onehot_encoded
        OneHot = myself_OneHotEncoder()
        onehot_encoded = OneHot.get_OneHotEncoder(labels)
        print('onehot_encoded',onehot_encoded)
        n_labels = onehot_encoded.shape[1]
        '''
        n_labels = labels.shape[1]
        p_classes = {}
        for i in range(n_labels):
            p_classes[i] = np.sum(labels[:,i])
              
        p_Num = {}
        p_Denom = {}
        for label in range(n_labels):
            for i in range(numTrainDocs):
                if labels[i][label] == 1:
                    #print('np.where(onehot_encoded[i]==1)[0]',np.where(onehot_encoded[i]==1)[0])
                    print(i)
                    if label not in p_Num.keys():
                        p_Num[label] = np.zeros(numWords)
                        p_Denom[label] = 0.0
                    p_Num[label] += train_Matrix[i]
                    p_Denom[label] += sum(train_Matrix[i])
                    #print("p_Num:{} '\n' p_Denom:{}".format(p_Num,p_Denom))
        
        p_Vec = {}
        for label in range(n_labels):
            if label not in p_Vec:
                p_Vec[label] = 0.0
            #log
            p_Vec[label] = np.log(p_Num[label]/p_Denom[label] + 1e-9)
            #p_Vec[label] = p_Num[label]/p_Denom[label] 
        return p_Vec,p_classes     
    
    
    #
    def classifyNB(self,vec2Classify, p_Vec, p_Classes):
        # 元素相乘 如果log是相加
        '''
        p_1 = sum(vec2Classify * p_1_Vec)  + np.log(pClass1)
        p_0 = sum(vec2Classify * p_0_Vec) + np.log(1.0 - pClass1)
        if p_1 > p_0:
            return 1
        else :
            return 0
        '''
        p = {}
        for label in p_Classes.keys():
            p[label] = sum(vec2Classify * p_Vec[label])  + np.log(p_Classes[label])
        p_max = Counter(p).most_common(2) #dict sorted by values. If you want get mulity result , you can set most_common(n)
        return p_max
        
    def testingNB(self,test_data,p_Vec, p_Classes):
        #myVocabList = createVocabList(test_data)
        test_Matrix =[]
        preds = []
        for testd in test_data:
            test_Matrix = self.setofWord2Vec(self.vocabSetlist,testd)
            pred = self.classifyNB(test_Matrix,p_Vec, p_Classes)
            preds.append(pred)
            
        return preds
        
        

#### 文本切分

In [ ]:
import re
def words_split(dataset):
    data_split = []
    regEx = re.compile('\\W*')
    for data in dataset:
        data_temp = [d.lower() for d in regEx.split(data) if len(d)>0]
        data_split.append(data_temp)
    return data_split

In [ ]:
words = ['my dog has flea, problems help please',
        'maybe not take him to dog park stupid',
        'my dalmation is so cute I love him',
        'stop posting stupid worthless garbage',
         ' mr licks ate my steak how to stop him',
        'quit buying worthless dog food stupid',
        'this noing i just look at it',
        'everything is ok']
words = words_split(words)
#classVec = [0,1,0,1,0,1,2,2]
words

In [ ]:
bayes = bayes()
myVocablist = bayes.createVocabList(words)
#myword2Vec = bayes.setofWord2Vec(myVocablist,words[0])
myword2Vec = []
for word_1 in words :
    myword2Vec.append(bayes.setofWord2Vec(myVocablist,word_1))
len(myVocablist)#,myword2Vec

In [ ]:
myVocablist

In [ ]:
classVec = np.array([[1,0,1],
 [0, 1, 0],
 [1, 0, 0],
 [0, 1, 1],
 [1, 0, 0],
 [0, 1, 0],
 [1, 0, 1],
 [0, 0 ,1]])

In [ ]:
#bayes = bayes()
p_Vec, p_classes = bayes.trainNB0(words,classVec)
p_Vec, p_classes

In [ ]:
preds = bayes.classifyNB(myword2Vec[4], p_Vec, p_classes)
preds

In [ ]:
test_data = ['love my dalmation','stupid dog']
test_data = words_split(test_data)
test_data

In [ ]:
preds = bayes.testingNB(test_data,p_Vec,p_classes)
preds